In [14]:
import networkx as nx
from IPython.display import Image
from IPython.core.display import HTML 
import kappy
from pysb.export import export
from indra.sources import trips
from indra.assemblers.pysb import PysbAssembler
from indra.assemblers.pysb.kappa_util import im_json_to_graph
from indra.explanation.model_checker import remove_im_params

In [15]:
def make_im(model):
        kappa = kappy.KappaStd()
        model_str = export(model, 'kappa')
        kappa.add_model_string(model_str)
        kappa.project_parse()
        imap = kappa.analyses_influence_map()
        graph = im_json_to_graph(imap)
        im_pgv = nx.nx_agraph.to_agraph(graph)
        return im_pgv

In [38]:
sentences = [
    # EGF->EGFR->GRB2->SOS1-> KRAS
    'EGF binds EGFR.',
    'EGFR bound to EGF binds EGFR.',
    'EGFR bound to EGFR binds GRB2.',
    'EGFR bound to GRB2 is degraded.',
    # SPRY4/SOS1 bind competitively to GRB2 on its SH3 site
    # Doesn't work:
    # -'SPRY4 binds GRB2 not bound to SOS1.',
    # -'Unphosphorylated SOS1 binds GRB2 not bound to SPRY4.',
    # -'SOS1 bound to GRB2 bound to EGFR activates KRAS.',
    # Alternative that leaves out SPRY4:
    'GRB2 bound to EGFR is active.',
    'SOS1 binds active GRB2.',
    'SOS1 bound to GRB2 activates KRAS.',
    # KRAS->CRAF/BRAF
    'Active KRAS binds BRAF and RAF1.',
    'BRAF bound to KRAS binds BRAF and RAF1.',
    'RAF1 bound to KRAS binds BRAF.',
    'Vemurafenib binds BRAF and RAF1.',
    'The BRAF-BRAF complex not bound to vemurafenib is active.',
    'The BRAF-RAF1 complex not bound to vemurafenib is active.',
    'The RAF1-BRAF complex not bound to vemurafenib is active.',    
    'BRAF V600E not bound to vemurafenib is active.',
    'Active BRAF phosphorylates MAP2K1.',
    'Active RAF1 phosphorylates MAP2K1.',
    'Phosphorylated MAP2K1 is active.',
    # MEK, DUSP-> ERK
    # Doesn't work:
    # - 'Selumetinib binds MAP2K1.',
    # - 'Phosphorylated MAP2K1 not bound to selumetinib phosphorylates MAPK1.'
    'Active MAP2K1 phosphorylates MAPK1.',
    'DUSP6 dephosphorylates MAPK1.',
    'Phosphorylated MAPK1 is active.',
    'Active MAPK1 increases DUSP6, EGFR, and SPRY4.',
]

In [39]:
#s = 'GRB2 bound to EGFR and not bound to SPRY4 is active.'
#tp = trips.process_text(s, service_endpoint='drum-dev')
#tp.statements

In [40]:
statements = []
for sentence in sentences:
    print("Processing '%s'" % sentence)
    tp = trips.process_text(sentence, service_endpoint='drum-dev')
    statements += tp.statements

Processing 'EGF binds EGFR.'
Processing 'EGFR bound to EGF binds EGFR.'
Processing 'EGFR bound to EGFR binds GRB2.'
Processing 'EGFR bound to GRB2 is degraded.'
Processing 'GRB2 bound to EGFR is active.'
Processing 'SOS1 binds active GRB2.'
Processing 'SOS1 bound to GRB2 activates KRAS.'
Processing 'Active KRAS binds BRAF and RAF1.'
Processing 'BRAF bound to KRAS binds BRAF and RAF1.'
Processing 'RAF1 bound to KRAS binds BRAF.'
Processing 'Vemurafenib binds BRAF and RAF1.'
Processing 'The BRAF-BRAF complex not bound to vemurafenib is active.'
Processing 'The BRAF-RAF1 complex not bound to vemurafenib is active.'
Processing 'The RAF1-BRAF complex not bound to vemurafenib is active.'
Processing 'BRAF V600E not bound to vemurafenib is active.'
Processing 'Active BRAF phosphorylates MAP2K1.'
Processing 'Active RAF1 phosphorylates MAP2K1.'
Processing 'Phosphorylated MAP2K1 is active.'
Processing 'Active MAP2K1 phosphorylates MAPK1.'
Processing 'DUSP6 dephosphorylates MAPK1.'
Processing 'Pho

In [41]:
statements

[Complex(EGF(), EGFR()),
 Complex(EGFR(bound: [EGF, True]), EGFR()),
 Complex(EGFR(bound: [EGFR, True]), GRB2()),
 DecreaseAmount(None, EGFR(bound: [GRB2, True])),
 ActiveForm(GRB2(bound: [EGFR, True]), activity, True),
 Complex(SOS1(), GRB2(activity)),
 Activation(SOS1(bound: [GRB2, True]), KRAS()),
 Complex(KRAS(activity), BRAF()),
 Complex(KRAS(activity), RAF1()),
 Complex(BRAF(bound: [KRAS, True]), BRAF()),
 Complex(BRAF(bound: [KRAS, True]), RAF1()),
 Complex(RAF1(bound: [KRAS, True]), BRAF()),
 Complex(VEMURAFENIB(), BRAF()),
 Complex(VEMURAFENIB(), RAF1()),
 ActiveForm(BRAF(bound: [BRAF, True], bound: [VEMURAFENIB, False]), activity, True),
 ActiveForm(BRAF(bound: [RAF1, True], bound: [VEMURAFENIB, False]), activity, True),
 ActiveForm(RAF1(bound: [BRAF, True], bound: [VEMURAFENIB, False]), activity, True),
 ActiveForm(BRAF(muts: (V, 600, E), bound: [VEMURAFENIB, False]), activity, True),
 Phosphorylation(BRAF(activity), MAP2K1()),
 Phosphorylation(RAF1(activity), MAP2K1()),
 Ac

In [42]:
pa = PysbAssembler(statements)
pa.make_model('two_step')

INFO: [2019-06-05 16:49:48] indra.assemblers.pysb.assembler - Using two_step assembly policy.


<Model 'indra_model' (monomers: 12, rules: 44, parameters: 57, expressions: 0, compartments: 0) at 0x1349eac88>

In [43]:
im = make_im(pa.model)
remove_im_params(pa.model, im)
im.draw('egf.png', prog='dot')

In [44]:
Image(url="./egf.png", width=400, unconfined=True)

In [45]:
from indra.tools import assemble_corpus as ac
ac.dump_statements(statements, 'marm_model_stmts.pkl')

INFO: [2019-06-05 16:49:55] indra.tools.assemble_corpus - Dumping 27 statements into marm_model_stmts.pkl...
